In [4]:
import scipy
import glob
import os
import librosa as lb
import matplotlib.pyplot as plt
import numpy as np
import math
import deepdish as dd
from scipy.stats import norm, zscore, pearsonr
import sys


def sliding_avg(data, rawData,sr):
    if len(data.shape) > 1:
        w = np.round(data.shape[1]/(len(rawData)/sr))
        output = np.zeros((data.shape[0], int(np.round((data.shape[1]/w)))))
        forward_idx = np.arange(0,data.shape[1] + 1, w)
        
    for i in range(len(forward_idx)):
        if data[:,int(forward_idx[i]):int(forward_idx[i])+int(w)].shape[1] != w:
            continue
        else:
            output[:,i] = np.mean(data[:,int(forward_idx[i]):int(forward_idx[i]) + int(w)],axis=1).T
    return output

path = '/home/msachs/music_emo_events/'
filepath = path + 'full_sequences_trimmed/'
dir_names =  [elem for elem in os.listdir(filepath) if '.mp3' in elem]
print(dir_names)

fname = path + '/emo_sums_36ss_03-18'
emo_sums = dd.io.load(fname)
studies = list(emo_sums.keys())
emos = ['calm','happy','sad','anxious','nostalgic']

['A2_1.mp3', 'B2_2.mp3', 'B1_3.mp3', 'B2_3.mp3', 'A1_1.mp3', 'B1_2.mp3', 'A1_2.mp3', 'B1_1.mp3', 'A2_3.mp3', 'A1_3.mp3', 'A2_2.mp3', 'B2_1.mp3']


In [3]:
def crosscor(v1,v2,cc_win,ignore_TRs):
    #v2 = v2[ignore_TRs:]
    cc = np.zeros(2 * cc_win + 1)
    for i in range(2 * cc_win + 1):
        if i == 2 * cc_win:
            v2_shift = v2[i:]
        else:
            v2_shift = v2[i:(-2 * cc_win + i)]
        cc[i] = pearsonr(v2_shift, v1[:(-2 * cc_win)])[0]
    #lag = np.arange(-1 * cc_win, cc_win + 1)[np.where(cc == max(cc))[0]][0]
    lag = np.where(cc == max(cc))[0]
    return lag,cc[np.argmax(cc)],cc

## Code from Williams - MFCC, CHROMA, Tempogram

In [ ]:
#wave files are 48 kHz with 24 bits per sample
dir_names.sort()
for i in dir_names[1:]:
    iname = i[-8:-4]
    audio_file = filepath + i
    
    ofile_temp = path + '/acoustic_features/' + iname + '_tempo'
    ofile_chroma = path + '/acoustic_features/' + iname + '_chroma'
    ofile_mfcc = path + '/acoustic_features/' + iname + '_mfcc'
    if os.path.exists(ofile_temp):
        continue

    # This is how we load audio using Librosa
    print(i,iname,'loading...')
    audio, sr = lb.load(audio_file, sr=44100, mono=False)
    #audio, sr = librosa.load(audio_file, sr=None)

    # Convert from stereo to mono since stft function only seems to work on mono data
    audio_mono = lb.core.to_mono(audio)
    print(len(audio_mono)/sr,sr)

    y_harm = lb.effects.harmonic(y=audio_mono, margin=8)
    chroma_os_harm = lb.feature.chroma_cqt(y=y_harm, sr=sr, bins_per_octave=12*3)
    chroma_filter = np.minimum(chroma_os_harm,lb.decompose.nn_filter(chroma_os_harm, aggregate=np.median, metric='cosine'))
    
    print(i,iname,'compressing...')
    chroma_smooth = sliding_avg(scipy.ndimage.median_filter(chroma_filter, size=(1, 9)),audio_mono,sr)

    # Compute MFCC then compress in time
    mfcc = lb.feature.mfcc(y=audio_mono, sr=sr)
    compressed_mfcc = sliding_avg(mfcc,audio_mono,sr)

    # Compute tempogram then compress in time
    tgram = lb.feature.tempogram(y=audio_mono, sr=sr)
    compressed_tgram = sliding_avg(tgram[1:,:],audio_mono,sr)
    
    print(compressed_tgram.shape,chroma_smooth.shape,compressed_mfcc.shape)

    np.save(ofile_temp, compressed_tgram) 
    np.save(ofile_chroma,chroma_smooth)
    np.save(ofile_mfcc,compressed_mfcc)

A1_2.mp3 A1_2 loading...


/home/msachs/.conda/envs/venv/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


256.4440816326531 44100


/home/msachs/.conda/envs/venv/lib/python3.6/site-packages/scipy/sparse/lil.py:512: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
/home/msachs/.conda/envs/venv/lib/python3.6/site-packages/scipy/sparse/lil.py:514: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):
/home/msachs/.conda/envs/venv/lib/python3.6/site-packages/scipy/sparse/lil.py:512: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
/home/msachs/.conda/envs/venv/lib/python3.6/site-packages/scipy/sparse/lil.py:518: FutureWarning: future versions will no

A1_2.mp3 A1_2 compressing...
(383, 257) (12, 257) (20, 257)
A1_3.mp3 A1_3 loading...


...

/home/msachs/.conda/envs/venv/lib/python3.6/site-packages/scipy/sparse/lil.py:512: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
/home/msachs/.conda/envs/venv/lib/python3.6/site-packages/scipy/sparse/lil.py:514: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):
/home/msachs/.conda/envs/venv/lib/python3.6/site-packages/scipy/sparse/lil.py:512: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
/home/msachs/.conda/envs/venv/lib/python3.6/site-packages/scipy/sparse/lil.py:518: FutureWarning: future versions will no

(383, 283) (12, 283) (20, 283)
B1_3.mp3 B1_3 loading...


/home/msachs/.conda/envs/venv/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


305.8938775510204 44100


/home/msachs/.conda/envs/venv/lib/python3.6/site-packages/scipy/sparse/lil.py:512: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
/home/msachs/.conda/envs/venv/lib/python3.6/site-packages/scipy/sparse/lil.py:514: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):
/home/msachs/.conda/envs/venv/lib/python3.6/site-packages/scipy/sparse/lil.py:512: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
/home/msachs/.conda/envs/venv/lib/python3.6/site-packages/scipy/sparse/lil.py:518: FutureWarning: future versions will no

## Cross correlation and pearson with emotional label sums

In [567]:
dir_names.sort()
cc_win = 10

corrs = {e: np.zeros((7,len(studies))) for e in emos}
for i,d in enumerate(dir_names):
    dname = d[-8:-4]
    study = [elem for elem in studies if dname.lower() in elem][0]
    
    ofile_temp = path + 'acoustic_features/' + dname + '_tempo.npy'
    ofile_chroma = path + 'acoustic_features/' + dname + '_chroma.npy'
    ofile_mfcc = path + 'acoustic_features/' + dname + '_mfcc.npy'
    if os.path.exists(ofile_temp):
        tgram_ds = np.load(ofile_temp) 
        chroma_ds = np.load(ofile_chroma)
        mfcc_ds = np.load(ofile_mfcc)
        print(dname,study, emo_sums[study][emos[0]].shape, mfcc_ds.shape, chroma_ds.shape,tgram_ds.shape)   
        if emo_sums[study][emos[0]].shape[0] == mfcc_ds.shape[1]:
            ignore_TRs = 0
        else:
            ignore_TRs = 1
        for e in emos:
            #CHROMA
            if ignore_TRs > 0:
                chroma_mean = chroma_ds[:,:-ignore_TRs].mean(axis = 0)
                chroma_std = chroma_ds[:,:-ignore_TRs].std(axis = 0)
                chroma_max = chroma_ds[:,:-ignore_TRs].max(axis = 0)
                tgram_mean = tgram_ds[:,:-ignore_TRs].mean(axis = 0)
                tgram_std = tgram_ds[:,:-ignore_TRs].std(axis = 0)
                tgram_max = tgram_ds[:,:-ignore_TRs].max(axis = 0)
                mfcc_one  = mfcc_ds[0,:-ignore_TRs]
            else:
                chroma_mean = chroma_ds.mean(axis = 0)
                chroma_std = chroma_ds.std(axis = 0)
                chroma_max = chroma_ds.max(axis = 0)
                tgram_mean = tgram_ds.mean(axis = 0)
                tgram_std = tgram_ds.std(axis = 0)
                tgram_max = tgram_ds.max(axis = 0)
                mfcc_one  = mfcc_ds[0]
                
            pR = pearsonr(emo_sums[study][e],chroma_mean)[0]
            pr_sd = pearsonr(emo_sums[study][e],chroma_std)[0]
            pr_max = pearsonr(emo_sums[study][e],chroma_max)[0]
            #print(e,'chroma mean:',pR,'chroma std:',pr_sd,'chroma max:',pr_max)
            
            pR2 = pearsonr(emo_sums[study][e],tgram_mean)[0]
            pr_sd2 = pearsonr(emo_sums[study][e],tgram_std)[0]
            pr_max2 = pearsonr(emo_sums[study][e],chroma_max)[0]
            #print(e,'tgram mean:',pR,'tgram std:',pr_sd,'tgram max',pr_max)
            
            pr_mfcc = pearsonr(emo_sums[study][e],mfcc_one)[0]
            #print(e,'mfcc/rms',pr_mfcc)
            corrs[e][:,i] = pR,pr_sd,pr_max,pR2,pr_sd2,pr_max2,pr_mfcc
            #corrs[e] = np.append([])
for e in emos:
    #print(e,corrs[e].max(axis = 1))
    print(e,corrs[e].mean(axis = 1))

    # FOR CORRELATIONS WITH ALL FEAUTRES (NOT MEAN, STD, or MAX)
#             for d in range(chroma_ds.shape[0]):
#                 lag,corr,cc = crosscor(emo_sums[study][e],chroma_ds[d,:],cc_win,ignore_TRs)
#                 if ignore_TRs > 0:
#                     pR = pearsonr(emo_sums[study][e],chroma_ds[d,:-ignore_TRs])[0]
#                 else:
#                     pR = pearsonr(emo_sums[study][e],chroma_ds[d,])[0]
                    
#                 if abs(corr) > 0.5:
#                     print('\t','chroma',d,e,corr,lag)
                
#                 if abs(pR) > 0.5:
#                     print('\t','full chroma',d,e,pR)
            
#              #MFCC
#             for d in range(mfcc_ds.shape[0]):
#                 lag,corr,cc = crosscor(emo_sums[study][e],mfcc_ds[d,:],cc_win,ignore_TRs)
#                 if ignore_TRs > 0:
#                     pR = pearsonr(emo_sums[study][e],mfcc_ds[d,:-ignore_TRs])[0]
#                 else:
#                     pR = pearsonr(emo_sums[study][e],mfcc_ds[d,])[0]
#                 if abs(corr) > 0.5:
#                     print('\t','mfcc',d,e,corr,lag)
#                 if abs(pR) > 0.5:
#                     print('\t','full mfcc',d,e,pR)
#             #TGRAM
#             for d in range(tgram_ds.shape[0]):
#                 lag,corr,cc = crosscor(emo_sums[study][e],tgram_ds[d,:],cc_win,ignore_TRs)
#             if ignore_TRs > 0:
#                     pR = pearsonr(emo_sums[study][e],tgram_ds[d,:-ignore_TRs])[0]
#                 else:
#                     pR = pearsonr(emo_sums[study][e],tgram_ds[d,])[0]
                    
#                 if abs(corr) > 0.5:
#                     print('\t','tgram',d,e,corr,lag)
#                 if abs(pR) > 0.5:
#                     print('\t','full tgram',d,e,pR)
# #                     plt.plot(np.arange(-1 * cc_win, cc_win + 1), cc)
# #                     plt.xlabel('Shift')
# #                     plt.ylabel('Correlation')
# #                     plt.title(d,e)
# #                     plt.show()

A1_1 musevents_a1_1 (323,) (20, 324) (12, 324) (383, 324)
A1_2 musevents_a1_2 (256,) (20, 257) (12, 257) (383, 257)
A1_3 musevents_a1_3 (315,) (20, 316) (12, 316) (383, 316)
A2_1 musevents_a2_1 (323,) (20, 324) (12, 324) (383, 324)
A2_2 musevents_a2_2 (257,) (20, 257) (12, 257) (383, 257)
A2_3 musevents_a2_3 (315,) (20, 316) (12, 316) (383, 316)
B1_1 musevents_b1_1 (307,) (20, 308) (12, 308) (383, 308)
B1_2 musevents_b1_2 (282,) (20, 283) (12, 283) (383, 283)
B1_3 musevents_b1_3 (305,) (20, 306) (12, 306) (383, 306)
B2_1 musevents_b2_1 (307,) (20, 308) (12, 308) (383, 308)
B2_2 musevents_b2_2 (288,) (20, 289) (12, 289) (383, 289)
B2_3 musevents_b2_3 (305,) (20, 306) (12, 306) (383, 306)
calm [-0.28512042  0.20791181  0.10781135  0.13483812  0.12210391  0.10781135
 -0.46728444]
happy [ 0.08785508 -0.09790866 -0.07903233 -0.09304129 -0.0864293  -0.07903233
  0.30259965]
sad [-0.28594844  0.19178031  0.10144243  0.16878761  0.15316106  0.10144243
 -0.35030908]
anxious [ 0.28510433 -0.1196

## Extract Additional features beyond tempogram, mfcc, and chroma (spectral, rms, tonal)

In [484]:
#tonal centroid --> key clarity --> standard deviation of tonez (standard deviation ) maybe? or to chroma cqt
#Centroid mean --> brigthness
#Rolloff --> also brightness 
#Spread -> MIRSPREAD --> fullness (wide or narrow) --> bandwith

feats_short = ['centroid','spread','rolloff','rms','specnovel','tonnetz']

#wave files are 48 kHz with 24 bits per sample
dir_names.sort()
for i in dir_names:
    iname = i[-8:-4]
    ofile = '%s/acoustic_features/%s_feats_nowin.npy' %(path,iname)
    audio_file = filepath + i
    y, sr = lb.load(audio_file,sr=None)

    win = int(window*sr)
    hop = int(hopsize*sr)
    length = len(y)/sr
    
    print('Computing features for',i)
    centroid = lb.feature.spectral_centroid(y,sr)
    spread = lb.feature.spectral_bandwidth(y,sr)
    rolloff = lb.feature.spectral_rolloff(y,sr)
    rms = lb.feature.rms(y)
    specnovel = lb.onset.onset_strength(y, sr=sr)
    tonnetz = lb.feature.tonnetz(y=y, sr=sr)

    short_feats = np.vstack([centroid,spread,rolloff,rms,specnovel,tonnetz]) 
    short_feats_ds = sliding_avg(short_feats,y,sr)
    
    ofile_mfcc = path + 'acoustic_features/' + iname + '_mfcc.npy'
    mfcc_ds = np.load(ofile_mfcc)
    print('RMS - MFCC 1', pearsonr(short_feats_ds[3,:],mfcc_ds[0,])[0])
    print('RMS - MFCC 2', pearsonr(short_feats_ds[3,:],mfcc_ds[1,])[0])
    
    if not os.path.exists(ofile):
        np.save(ofile,short_feats_ds)

for c in chroma_ds.shape[0]:
    print(pearsonr(chroma_ds[c],tonnetz))

/home/msachs/.conda/envs/venv/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


Computing features for A1_1.mp3


/home/msachs/.conda/envs/venv/lib/python3.6/site-packages/scipy/sparse/lil.py:512: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
/home/msachs/.conda/envs/venv/lib/python3.6/site-packages/scipy/sparse/lil.py:514: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):


RMS - MFCC 1 0.7888616664349981
RMS - MFCC 2 0.5256659348971483


/home/msachs/.conda/envs/venv/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


...

RMS - MFCC 1 0.8155792313951734
RMS - MFCC 2 0.41789407402722667


/home/msachs/.conda/envs/venv/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


Computing features for B2_3.mp3


/home/msachs/.conda/envs/venv/lib/python3.6/site-packages/scipy/sparse/lil.py:512: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
/home/msachs/.conda/envs/venv/lib/python3.6/site-packages/scipy/sparse/lil.py:514: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):


RMS - MFCC 1 0.8156701728267645
RMS - MFCC 2 0.6240788937118451


In [ ]:
## Correlate spectral, rms, tonnetz with ratings

In [38]:
fname = path + '/emo_sums_36ss_03-18'
emo_sums = dd.io.load(fname)
studies = list(emo_sums.keys())
emos = ['calm','happy','sad','anxious','nostalgic']
dir_names.sort()
cc_win = 8
corrs = {e: np.zeros((len(feats_short),len(studies))) for e in emos}

for i,d in enumerate(dir_names):
    dname = d[-8:-4]
    study = [elem for elem in studies if dname.lower() in elem][0]
    ofile = '%s/acoustic_features/%s_feats_nowin.npy' %(path,dname)
    #ofile = '%s/acoustic_features/%s_short_feats.npy' %(path,dname)
    if os.path.exists(ofile):
        short_feats = np.load(ofile)  
        
    if emo_sums[study][emos[0]].shape[0] == short_feats.shape[1]:
        ignore_TRs = 0
    else:
        ignore_TRs = 1
    emosums = np.zeros(emo_sums[study][emos[0]].shape)
    for enum in range(len(emos)+1):
        if enum != len(emos):
            e = emos[enum]
            emovec = emo_sums[study][e]
            emosums += emo_sums[study][e]
            
        else:
            e = 'sum'
            emovec = emosums
            
        for f, feat in enumerate(feats_short): 
            if ignore_TRs > 0:
                featvec = short_feats[f,:-ignore_TRs]
            else:
                featvec = short_feats[f,:]
            
            if feat == 'tonnetz':
                if ignore_TRs > 0:
                    featvec = short_feats[f:f+6,:-ignore_TRs].max(axis =0)
                else:
                    featvec = short_feats[f:f+6,:].max(axis =0)
            
            pR = pearsonr(emovec,featvec)[0]
            lag,corr,cc = crosscor(featvec,emovec,cc_win,ignore_TRs)
            
            if e != 'sum':
                corrs[e][f,i] = pR

            if abs(corr) > 0.5:
                print('\t',d,e,feat,corr,lag)

            if abs(pR) > 0.5:
                print('\t',d,e,feat,pR)
for e in emos:
    print(e,corrs[e].mean(axis =1))

	 A1_1.mp3 sum centroid -0.1118962773613967
	 A1_1.mp3 sum centroid -0.10949122597829188 [0]
	 A1_1.mp3 sum spread -0.05393273088748676
	 A1_1.mp3 sum spread -0.08474666014449181 [0]
	 A1_1.mp3 sum rolloff -0.09609065385370882
	 A1_1.mp3 sum rolloff -0.07977868345179429 [0]
	 A1_1.mp3 sum rms 0.25534216507456475
	 A1_1.mp3 sum rms 0.4372440449224152 [0]
	 A1_1.mp3 sum specnovel -0.023307273890310412
	 A1_1.mp3 sum specnovel 0.07253869498798253 [0]
	 A1_1.mp3 sum tonnetz 0.012258834267481025
	 A1_1.mp3 sum tonnetz -0.044361447707938706 [13]
	 A1_2.mp3 sum centroid 0.20212255816109925
	 A1_2.mp3 sum centroid 0.18963277473412363 [0]
	 A1_2.mp3 sum spread 0.17571818861007724
	 A1_2.mp3 sum spread 0.18200936029913092 [0]
	 A1_2.mp3 sum rolloff 0.15559689621857034
	 A1_2.mp3 sum rolloff 0.15309002416367848 [0]
	 A1_2.mp3 sum rms 0.16468204673131193
	 A1_2.mp3 sum rms 0.1833422812348758 [1]
	 A1_2.mp3 sum specnovel 0.06667071757081798
	 A1_2.mp3 sum specnovel 0.08321165836940024 [3]
	 A1_2.mp

## Tempo and beat

In [692]:
window = 3 #inms
hopsize = window*.67

dir_names.sort()
beat_corrs = np.zeros((len(emos),len(studies)))
for inum,i in enumerate(dir_names[1:]):
    iname = i[-8:-4]
    audio_file = filepath + i
    print(i,iname,'loading...')
    x, sr = lb.load(audio_file, sr=None, mono=False)
    xmono = lb.core.to_mono(x)
    #print(len(xmono)/sr,sr)
    
#     win = int(window*sr)
#     hop = int(hopsize*sr)
#     length = len(y)/sr
    
#     #get tempo
#     tempo = lb.beat.tempo(xmono, sr=sr, start_bpm = 80, max_tempo = 160)
#     print(i,iname,tempo)
    
#     # estimated beat event locations in the specified units (default is frame indices)
#     onset_env = lb.onset.onset_strength(xmono, sr=sr)
#     frames = range(len(onset_env))
    
    t, beats = lb.beat.beat_track(xmono, sr=sr,start_bpm=80,bpm = 80,units='time')
    beats_diff = np.diff(beats)
    beats_ds = np.zeros((round(len(xmono)/float(sr))))
    for s in range(round(len(xmono)/float(sr))):
        idcs = np.where((beats > s) & (beats < s+1))[0]
        try:
            beats_ds[s] = beats_diff[idcs].mean()
            #print(s,idcs,beats_diff[idcs])
        except IndexError:
            beats_ds[s] = beats_diff[idcs-1].mean()
            #print(s,idcs,beats_diff[idcs-1])
    
    #correlate with ratings
    study = [elem for elem in studies if iname.lower() in elem][0]  
    v1 = beats_ds[~np.isnan(beats_ds)]
    for enum,e in enumerate(emos):
        v2 = emo_sums[study][e][0:len(v1)]
        pR = pearsonr(v2,v1)[0]
        lag,corr,cc = crosscor(v1,v2,cc_win,ignore_TRs)
        beat_corrs[enum,inum] = pR

        if abs(corr) > 0.3:
            print('\t',d,e,corr,lag)

        if abs(pR) > 0.3:
            print('\t',d,e,pR)
#for e in emos:
    #print(e,corrs[e].mean(axis =1))

SyntaxError: invalid syntax (<ipython-input-692-7e4c0d26d00b>, line 70)

## Extracting features in predefined frames (beta - need to update)

In [ ]:
feats_long = ['tonez','chroma']
feats_short = 
#window = 0.025 #inms
#hopsize = window*.5

window = 3 #inms
hopsize = window*.67
dir_names.sort()
for i in dir_names:
    win = int(window*sr)
    hop = int(hopsize*sr)
    
    iname = i[-8:-4]
    ofile = '%s/acoustic_features/%s_short_feats.npy' %(path,iname)
    audio_file = filepath + i
    y, sr = lb.load(audio_file,sr=None)
    tonnetz_all = lb.feature.tonnetz(y=y, sr=sr)
    tonnetz_ds = sliding_avg(tonnetz_all,y,sr)
    
    fig, ax = plt.subplots(figsize = (18,5),nrows=1)
    #plt.figure(figsize=(18,5)) 
    img1 = lb.display.specshow(tonnetz_all,y_axis='tonnetz', x_axis='time', ax=ax, sr = sr)
    ax.set(title='Tonal Centroids (Tonnetz)')
    ax.label_outer()

    
    yframe = lb.util.frame(y, win, hop, axis=0)
    nframe = yframe.shape[0]
    tonnetz_frames = []
    keyclarity = np.zeros((259*nframe))
    tonnetz_all2 = np.zeros((6,259*nframe))
    for f in range(yframe.shape[0]):
        #tonnetz_frames.append(lb.feature.tonnetz(y=yframe[f], sr=sr))
        print(f,tonnetz_frames[f].shape, tonnetz_frames[f].std(), tonnetz_frames[f].mean())
        tonnetz_all2[:,(259*f):(259*f) + 259] = tonnetz_frames[f]
        
    tonnetz_ds2 = sliding_avg(tonnetz_all2,y,sr)

    fig, ax = plt.subplots(figsize = (16,10),nrows=2,sharex=True)
    img1 = lb.display.specshow(tonnetz_ds,y_axis='tonnetz', x_axis='time', ax=ax[0], sr = sr)
    ax[0].set(title='Tonal Centroids (Tonnetz)')
    ax[0].label_outer()
    img2 = lb.display.specshow(tonnetz_ds2,y_axis='tonnetz', x_axis='time',ax=ax[1],sr = sr)
    ax[1].set(title='Tonal Centroids (Tonnetz)')
    #fig.colorbar(img1, ax=[ax[0]])
    #fig.colorbar(img2, ax=[ax[1]])

## Extras

In [ ]:
# Plot tempo and beats

    T = len(x)/float(sr)
    seconds_per_beat = 60.0/tempo[0]
    beat_times = np.arange(0, T, seconds_per_beat)
    lb.display.waveplot(x, sr = sr)
    
    plt.vlines(beat_times, -1, 1, color='r')
    
    clicks = librosa.clicks(beat_times, sr, length=len(x))
    ipd.Audio(x + clicks, rate=sr)
    
    plt.figure(figsize=(18,5)) 
    tempogram = lb.feature.tempogram(onset_envelope=onset_env, sr=sr, hop_length=hop, win_length=win)
    
    lb.display.specshow(tempogram, sr=sr, hop_length=hop, x_axis='time', y_axis='tempo')
    pulse_clarity = 
    
    
    plt.figure(figsize=(14, 5))
    lb.display.waveplot(xmono,sr =sr,alpha=0.6)
    plt.vlines(beats, -1, 1, color='r')
    plt.ylim(-1, 1)